<a href="https://colab.research.google.com/github/thiswind/machine-learning-study/blob/main/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!nvidia-smi
!pwd

Wed Nov  9 16:26:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [39]:
from google.colab import drive
drive.mount('/content/drive/')
!ln -s drive/MyDrive/colab $(pwd)/colab
!mkdir -p ./colab/gan
DRIVE_DIR = './colab/gan/'
!ls -l
!ls -l ./colab/gan

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
ln: failed to create symbolic link '/content/colab/colab': File exists
total 1668632
lrwxrwxrwx 1 root root        19 Nov  9 15:40 colab -> drive/MyDrive/colab
-rw-r--r-- 1 root root 853083403 Nov  9 15:56 dogs-vs-cats-redux-kernels-edition.zip
drwx------ 5 root root      4096 Nov  9 15:40 drive
drwxr-xr-x 1 root root      4096 Nov  7 14:37 sample_data
-rw-r--r-- 1 root root    113903 Dec 11  2019 sample_submission.csv
drwxr-xr-x 2 root root    286720 Sep 20  2013 test
-rw-r--r-- 1 root root 284478493 Dec 11  2019 test.zip
drwxr-xr-x 2 root root    770048 Sep 20  2013 train
-rw-r--r-- 1 root root 569918665 Dec 11  2019 train.zip
total 0


In [43]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"windthis","key":"a968dadd8de108941769a83c92588874"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!rm -f *.zip
!rm -f *.csv
!rm -rf ./train/
!rm -rf ./test/
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition
!unzip -q dogs-vs-cats-redux-kernels-edition.zip
!unzip -q ./train.zip
!unzip -q ./test.zip

100% 813M/814M [00:22<00:00, 42.1MB/s]
100% 814M/814M [00:22<00:00, 38.7MB/s]


In [41]:
# 导入开发需要的库
import keras
import os
import shutil
import threading
import numpy as np
import cv2
import h5py

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from keras.models import *
from keras.layers import *

from keras.applications import *
from keras.preprocessing import *
from keras.preprocessing.image import *


In [44]:
def load_data(load_type="train"):
    path = None
    n = 25000
    if load_type=="train":
        imgs = []
        labels = []
        
        path = "./train/"
        img_names = os.listdir(path)
        
        for name in img_names:
            imgs.append("train/"+name)
            labels.append([0] if name[:3] == "cat" else [1])
            
        train_img_names,valid_img_names,train_labels,valid_labels = train_test_split(imgs,labels,test_size=0.2,random_state=42)
        return train_img_names,valid_img_names,train_labels,valid_labels
    else:
        # test,don`t have the labels
        path = "./test"
        img_names = os.listdir(path)
        imgs = []
        for img in img_names:
            imgs.append(img)
                
        return imgs


In [45]:
train_img_names,valid_img_names,train_labels,valid_labels = load_data()

In [46]:
print(train_img_names[:5])
print(train_labels[:5])
print(len(train_img_names)+len(valid_img_names))

['train/dog.2474.jpg', 'train/cat.11618.jpg', 'train/cat.12202.jpg', 'train/dog.4031.jpg', 'train/cat.5840.jpg']
[[1], [0], [0], [1], [0]]
25000


In [47]:
# 定义keras的generator方法
# custom keras generator
class MOGenerator(keras.utils.Sequence):
    def __init__(self, data, n, des_size=(224, 224), means=None, stds=None,
                 is_directory=True, batch_size=32, shuffle=True, seed=0):
        '''
        data: tuple of (x,y)
        n: data size
        des_size: standard size
        means: the dataset mean of RGB,default is imagenet means [103.939, 116.779, 123.68]
        batch_size: default is 32
        shuffle: random the data,default is True
        '''
        self.x = np.array(data[0])
        if len(data) >= 2:
            self.y = np.array(data[1])
        else:
            self.y = None
        self.n = n
        self.des_size = des_size
        self.is_directory = is_directory
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.lock = threading.Lock()
        self.index_array = self._set_index_array()
        self.means = means
        self.stds = stds

    def reset_index(self):
        self.batch_index = 0

    def _set_index_array(self):
        self.index_array = np.arange(self.n)
        if self.shuffle:
            np.random.shuffle(self.index_array)

    def on_epoch_end(self):
        # 重置索引数组
        self._set_index_array()

    def __len__(self):
        # 计算batch的总数量
        return int(np.ceil(self.n / self.batch_size))

    def __getitem__(self, idx):
        if self.index_array is None:
            self._set_index_array()
        index_array = self.index_array[self.batch_size * idx:
                                       self.batch_size * (idx + 1)]
        return self._data_generate(index_array)

    def _data_generate(self, index_array):
        # read from path
        # request the memory
        imgs = np.zeros((len(index_array), self.des_size[0], self.des_size[1], 3), dtype=np.uint8)
        # read the data
        if self.is_directory:
            img_names = self.x[index_array]
            for name_index in range(len(img_names)): 
                img = cv2.imread(img_names[name_index])
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, self.des_size)
                    imgs[name_index] = img
        else:
            for i in range(len(index_array)):
                img = self.x[index_array[i]]
                img = cv2.resize(img, self.des_size)
                imgs[i] = img
        if self.y is not None:
            labels = self.y[index_array]
        if labels is None:
            return imgs
        else:
            return imgs, labels



In [48]:
batch_size = 32

train_img_names,valid_img_names,train_labels,valid_labels = load_data()

test_img_names = load_data(load_type="test")

train_generator_224 = MOGenerator((train_img_names,train_labels), len(train_img_names), des_size=(224,224),
                                  batch_size=batch_size, shuffle=True)

train_generator_299 = MOGenerator((train_img_names,train_labels), len(train_img_names), des_size=(299,299),
                                  batch_size=batch_size, shuffle=True)

valid_generator_299 = MOGenerator((valid_img_names,valid_labels), len(valid_img_names), des_size=(299,299),
                                  batch_size=batch_size, shuffle=False)

valid_generator_224 = MOGenerator((valid_img_names,valid_labels), len(valid_img_names), des_size=(224,224),
                                  batch_size=batch_size, shuffle=True)

test_generator_299 = MOGenerator((test_img_names), len(test_img_names), des_size=(299,299),
                                  batch_size=batch_size, shuffle=False)

test_generator_224 = MOGenerator((test_img_names), len(test_img_names), des_size=(224,224),
                                  batch_size=batch_size, shuffle=False)


In [53]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import resnet50

In [55]:
# 加载Resnet50网络, Include_top: 是否包含卷积之后的全连接层.
base_model = ResNet50(input_tensor=Lambda(resnet50.preprocess_input)(Input(shape=(224,224,3))), 
                      weights="imagenet", include_top=False)
# 遍历所有层, 将预训练模型的卷积层设置为不可训练, 这样它们的参数就不会被改变了.
for layers in base_model.layers:
    layers.trainable = False

# 重新设置输出层, 我们的类别是两类, 只需要一个神经元即可.
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)

# 实例化模型
model = Model(base_model.input, x)

# 设置优化器, 损失函数, 展示参数信息
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# 开始训练, 通过设置迭代器模式
model.fit_generator(train_generator_224,len(train_img_names)//batch_size,epochs=5,
                    validation_data=valid_generator_224,validation_steps=len(valid_img_names)//batch_size,shuffle=False)


Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


625/625 [==============================] - 90s 140ms/step - loss: 0.0540 - accuracy: 0.9812 - val_loss: 0.0369 - val_accuracy: 0.9860
Epoch 2/5
625/625 [==============================] - 94s 151ms/step - loss: 0.0321 - accuracy: 0.9884 - val_loss: 0.0348 - val_accuracy: 0.9882
Epoch 3/5
625/625 [==============================] - 95s 152ms/step - loss: 0.0264 - accuracy: 0.9904 - val_loss: 0.0383 - val_accuracy: 0.9878
Epoch 4/5
625/625 [==============================] - 85s 136ms/step - loss: 0.0254 - accuracy: 0.9909 - val_loss: 0.0376 - val_accuracy: 0.9878
Epoch 5/5
625/625 [==============================] - 86s 138ms/step - loss: 0.0239 - accuracy: 0.9913 - val_loss: 0.0343 - val_accuracy: 0.9880


In [ ]:
## inception
inception = inception_v3.InceptionV3(include_top=False,
        weights="imagenet",input_tensor=Lambda(inception_v3.preprocess_input)(Input(shape=(299,299,3))),pooling="avg")
output = inception.output

output = Dropout(0.25)(output)

prediction = Dense(1,activation="sigmoid")(output)

inception_model = Model(inputs=inception.input,outputs=prediction)

for layer in inception.layers: #[:-res_global_pool_index]
    layer.trainable = False

inception_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

inception_model.fit_generator(train_generator_299,(len(train_img_names) + batch_size - 1)//batch_size,epochs=5,
                       validation_data=valid_generator_299,validation_steps=len(valid_img_names)//batch_size)


87910968/87910968 [==============================] - 3s 0us/step
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


157/625 [======>.......................] - ETA: 58s - loss: 0.1372 - accuracy: 0.9640

In [ ]:
## xception
xcep = Xception(include_top=False, weights="imagenet", 
                input_tensor=Lambda(xception.preprocess_input)(Input(shape=(299,299,3))),pooling="avg")
output = xcep.output

output = Dropout(0.25)(output)

prediction = Dense(1,activation="sigmoid")(output)

xcep_model = Model(inputs=xcep.input,outputs=prediction)

for layer in xcep.layers:
    layer.trainable = False

xcep_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

xcep_model.fit_generator(train_generator_299,(len(train_img_names) + batch_size - 1)//batch_size,epochs=5,
                       validation_data=valid_generator_299,validation_steps=len(valid_img_names)//batch_size)


In [ ]:
model.save_weights("resnet.h5")
model.save_weights("xcep.h5")
model.save_weights("incep.h5")


In [ ]:
from keras.preprocessing.image import *

# 获取所有训练集的图片名
train_filenames = os.listdir("train")
test_filenames = os.listdir("test")

# 文件名是特殊命名形式的, 所以直接获取分类后的文件
train_cat = list(filter(lambda x:x[:3] == 'cat', train_filenames))
train_dog = list(filter(lambda x:x[:3] == 'dog', train_filenames))
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

# 创建目录
rmrf_mkdir('train2')
os.mkdir('train2/cat')
os.mkdir('train2/dog')

rmrf_mkdir('valid2')
os.mkdir('valid2/cat')
os.mkdir('valid2/dog')

rmrf_mkdir('test2')
os.mkdir("test2/test")
for filename in test_filenames:
  # 建立软链（不会从创建文件, 有点类似快捷方式）
  os.symlink("/content/test/"+filename, "/content/test2/test/"+filename)

for filename in train_cat[:-2500]:
  os.symlink("/content/train/"+filename, "/content/train2/cat/"+filename)

for filename in train_dog[:-2500]:
  os.symlink("/content/train/"+filename, "/content/train2/dog/"+filename)

for filename in train_cat[-2500:]:
  os.symlink("/content/train/"+filename, "/content/valid2/cat/"+filename)
    
for filename in train_dog[-2500:]:
  os.symlink("/content/train/"+filename, "/content/valid2/dog/"+filename)
    
# 使用Keras默认的生成器, 因为我们已经生成对应的目录了
# 224 是 resnet的处理方式, 299是xception, inception的处理大小
gen = ImageDataGenerator()

train_generator_224 = gen.flow_from_directory("train2", (224,224), shuffle=False, 
                                              batch_size=batch_size,class_mode='binary')

valid_generator_224 = gen.flow_from_directory("valid2", (224,224), shuffle=False, 
                                              batch_size=batch_size,class_mode='binary')

test_generator_224 = gen.flow_from_directory("test2", (224,224), shuffle=False, 
                                             batch_size=batch_size, class_mode=None)

train_generator_299 = gen.flow_from_directory("train2", (299,299), shuffle=False, 
                                              batch_size=batch_size,class_mode='binary')

valid_generator_299 = gen.flow_from_directory("valid2", (299,299), shuffle=False, 
                                              batch_size=batch_size,class_mode='binary')

test_generator_299 = gen.flow_from_directory("test2", (299,299), shuffle=False, 
                                             batch_size=batch_size, class_mode=None)


In [ ]:
import h5py

# 特征向量的保存方法
def write_feature(model_name,model,train_generator,train_labels,valid_generator,valid_labels,test_generator,batch_size=32):
    # 通过模型名字来加载不同的权重
    if model_name == 'resnet_feature':
        model.load_weights('resnet.h5',by_name=True)
    elif model_name == 'inception_feature':
        model.load_weights('incep.h5',by_name=True)
    else:
        model.load_weights('xcep.h5',by_name=True)
    # 转换为numpy数组
    train_labels = np.array(train_labels)
    valid_labels = np.array(valid_labels)
    # 直接进行输出, 得到特征向量
    train_feature = model.predict_generator(train_generator,int(np.ceil(train_generator.samples/batch_size)),verbose=1)
    valid_feature = model.predict_generator(valid_generator,int(np.ceil(valid_generator.samples/batch_size)),verbose=1)
    test_feature  = model.predict_generator(test_generator,int(np.ceil(test_generator.samples/batch_size)),verbose=1)
    print("train_feature.shape:",train_feature.shape)
    print("valid_feature.shape:",valid_feature.shape)
    # 保存到本地
    with h5py.File(model_name+'.h5','w') as file:
        file.create_dataset("train",data=train_feature,dtype="float32")
        file.create_dataset('trian_labels',data=np.array(train_generator.classes),dtype="uint8")
        file.create_dataset("valid",data=valid_feature,dtype="float32")
        file.create_dataset("valid_labels",data=np.array(valid_generator.classes),dtype="uint8")
        file.create_dataset("test",data=test_feature,dtype="float32")

        


In [ ]:
# resnet50
write_feature('resnet_feature',Model(inputs=model.input,outputs=model.layers[-3].output),
              train_generator_224,train_labels,valid_generator_224,valid_labels,test_generator_224)


In [ ]:
# inception
write_feature('inception_feature',Model(inputs=inception_model.input,outputs=inception_model.layers[-3].output),
              train_generator_299,train_labels, valid_generator_299,valid_labels,test_generator_299)



In [ ]:
# xception
write_feature('xception_feature',Model(inputs=xcep_model.input,outputs=xcep_model.layers[-3].output),
              train_generator_299,train_labels,valid_generator_299,valid_labels,test_generator_299)


In [ ]:
feature_files = ['resnet_feature.h5','inception_feature.h5','xception_feature.h5']

X_train = []
y_train = []
X_valid = []
y_valid = []
X_test = []

for file_name in feature_files:
    with h5py.File(file_name, 'r') as h:
        X_train.append(np.array(h['train']))
        X_valid.append(np.array(h['valid']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['trian_labels'])
        y_valid = np.array(h['valid_labels'])
        print(np.array(h['train']).shape,np.array(h['valid']).shape,np.array(h['test']).shape)

X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)
X_test = np.concatenate(X_test, axis=1)

print("last:",X_train.shape,X_valid.shape,X_test.shape)


In [ ]:
from sklearn.utils import shuffle
# 打乱数据，模拟现实中的随机效果
X_train, y_train = shuffle(X_train, y_train)


In [ ]:
import keras.utils

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
concatenate_model = Model(inputs=input_tensor, outputs=x)

concatenate_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

concatenate_model.fit(X_train,y_train,batch_size=128, epochs=5,validation_data=(X_valid,y_valid))#validation_split=0.2


In [ ]:
import pandas as pd
y_pred = concatenate_model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

df = pd.read_csv("sample_submission.csv")

image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("/content/test2/", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('sample_submission.csv', index=None)

print(df.head(20))


In [ ]:
!kaggle competitions submit -c dogs-vs-cats-redux-kernels-edition -f sample_submission.csv -m "Message"
